In [3]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

Setup was successful.


In [4]:
exchange = Exchange()
_ = exchange.connect()

2024-12-07 09:25:25,873 [asyncio   ] [MainThread  ] Using selector: EpollSelector


# Instrument Details

In [5]:
# Load all instruments (products) available on the exchange
instruments = exchange.get_instruments()
instruments

{'PHILIPS_B': Instrument(instrument_id=PHILIPS_B, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, paused=False, expired=False),
 'PHILIPS_A': Instrument(instrument_id=PHILIPS_A, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, paused=False, expired=False)}

In [8]:
# The returned type of exchange.get_instruments() is a dictionary, select a single instrument
instrument = instruments['PHILIPS_A']
print(instrument)

Instrument(instrument_id=PHILIPS_A, tick_size=0.1, price_change_limit=PriceChangeLimit(absolute_change=5.0000, relative_change=10.00%), instrument_type=InstrumentType.STOCK, paused=False, expired=False)


In [9]:
# From any instruments we can get a lot of data:
print(instrument.instrument_id)
print(instrument.instrument_type)
print(instrument.tick_size)

# For particular types of instruments, additional fields are populated (such as e.g. the option expiry, strike and kind, or the future expiry)
# For a STOCK those fields are equal to None
print(instrument.expiry)

print(instrument.option_kind)
print(instrument.strike)
print(instrument.base_instrument_id)

PHILIPS_A
InstrumentType.STOCK
0.1
None
None
None
None


In [10]:
# The InstrumentType is an Enum (enumerable) type, that means its value is one of a few supported categories.
# We can use this Enum to compare which specific type of Instrument we are dealing with.

# First import the InstrumentType enum (we would normally do this at the top of the file)
from optibook.common_types import InstrumentType

# Then make a comparison as such:
if instrument.instrument_type == InstrumentType.STOCK:
    print('It is a STOCK.')
elif instrument.instrument_type == InstrumentType.STOCK_OPTION:
    print('It is a STOCK_OPTION.')
elif instrument.instrument_type == InstrumentType.STOCK_FUTURE:
    print('It is a STOCK_FUTURE.')
else:
    print(f'It was another instrument_type: {instrument.instrument_type}.')

It is a STOCK.


# Order Book

In [21]:
# The exchange.get_last_price_book() method returns a large compound datatype storing bids and offers, with price and volumes, and a timestamp. 
book = exchange.get_last_price_book('PHILIPS_A')
book

PriceBook(PHILIPS_A 2024-12-07 09:08:10.080964)
 #bids | price | #asks 
       |  75.3 | 10000 
       |  69.3 |  250  
       |  69.2 |  400  
  650  |  68.7 |       
 10000 |  62.7 |       

In [22]:
# How do we take it apart? First, extract one side of the order book, the bids, say
bids = book.bids
bids

[PriceVolume(price=68.7, volume=650), PriceVolume(price=62.7, volume=10000)]

In [23]:
# That's a list of PriceVolume combinations, which makes sense, one line of the order book is always a price with corresponding volume and we have multiple such orders
# Let's find the most competitive order, which is always the first entry in the list
best_bid = bids[0]
best_bid

PriceVolume(price=68.7, volume=650)

In [24]:
# That's still a compound of price and volume, we can simply select which one we want to see to finally end up with a numeric type
print(best_bid.price)
print(best_bid.volume)

68.7
650


In [25]:
# Putting it all together we can also write this. 
book.bids[0].price

68.7

In [26]:
# Note you will see an error on the above line if there are no bids at all available (the list will have no entry 0)
# Better is to check first
if book.bids:
    print(book.bids[0].price)
else:
    print('No bids in the order book at all.')

68.7


# TradeTicks

In [28]:
# We can load the tradetick history from the exchange (since we connected), returning a list of TradeTick types
tradeticks = exchange.get_trade_tick_history('PHILIPS_A')

# That list might be quite long:
print(len(tradeticks))

# So let's print only the last 5:
last_tradeticks = tradeticks[-5:]
last_tradeticks

100


[TradeTick(timestamp=2024-12-07 09:10:34.065393, instrument_id=PHILIPS_A, price=69.9, volume=250, aggressor_side=ask, buyer=, seller=, trade_id=111899),
 TradeTick(timestamp=2024-12-07 09:10:34.694749, instrument_id=PHILIPS_A, price=70.4, volume=5, aggressor_side=bid, buyer=, seller=, trade_id=111900),
 TradeTick(timestamp=2024-12-07 09:10:35.209090, instrument_id=PHILIPS_A, price=70.5, volume=1, aggressor_side=bid, buyer=, seller=, trade_id=111902),
 TradeTick(timestamp=2024-12-07 09:10:36.693707, instrument_id=PHILIPS_A, price=70.5, volume=1, aggressor_side=bid, buyer=, seller=, trade_id=111905),
 TradeTick(timestamp=2024-12-07 09:10:39.089624, instrument_id=PHILIPS_A, price=70.3, volume=250, aggressor_side=ask, buyer=, seller=, trade_id=111909)]

In [18]:
# We can extract information of each indidividual tradetick by obtaining a specific TradeTick as an entry in the list:
last_tradetick = tradeticks[-1]
print(last_tradetick.timestamp)
print(last_tradetick.instrument_id)
print(last_tradetick.price)
print(last_tradetick.volume)

2022-08-08 14:06:32.883561
DEMO
86.60000000000001
100


In [30]:
# Let's say we wanted to calculate the average price of all tradeticks, we could use a loop for that
sum_of_prices = 0.0
for tradetick in tradeticks:
    sum_of_prices += tradetick.price
    
average_price = sum_of_prices / len(tradeticks)
print(average_price)

69.76599999999998


# Figuring out more

In [31]:
# We can load the documentation of any function via the help()-function
# This is also all available in a nice overview on https://{your-course}.optibook.net/docs?page=optibook
help(exchange.get_positions_and_cash)

Help on method get_positions_and_cash in module optibook.synchronous_client:

get_positions_and_cash() -> Dict[str, Dict] method of optibook.synchronous_client.Exchange instance
    Get your current positions and cash.
    
    Returns
    -------
    typing.Dict[str, typing.Dict]
        Returns a dictionary mapping instrument_id to dictionary of 'volume' and 'cash'. The volume is the
        current amount of lots held in the instrument and the cash is the current cash position arising from
        previous buy and sell trades in the instrument.



In [32]:
# Note the returned value desribed at the bottom of the function-documentation; indeed we see nested dictionaries as described
positions_and_cash = exchange.get_positions_and_cash()
positions_and_cash

{'PHILIPS_A': {'volume': 0, 'cash': 0.0},
 'PHILIPS_B': {'volume': 0, 'cash': 0.0}}

In [22]:
# For the other functions and data types, similar documentation is available:
help(InstrumentType)

Help on class InstrumentType in module optibook.common_types:

class InstrumentType(enum.Enum)
 |  An enumeration.
 |  
 |  Method resolution order:
 |      InstrumentType
 |      enum.Enum
 |      builtins.object
 |  
 |  Data and other attributes defined here:
 |  
 |  INDEX_FUTURE = <InstrumentType.INDEX_FUTURE: 6>
 |  
 |  INDEX_OPTION = <InstrumentType.INDEX_OPTION: 5>
 |  
 |  INDEX_TRACKING_ETF = <InstrumentType.INDEX_TRACKING_ETF: 4>
 |  
 |  STOCK = <InstrumentType.STOCK: 1>
 |  
 |  STOCK_FUTURE = <InstrumentType.STOCK_FUTURE: 3>
 |  
 |  STOCK_OPTION = <InstrumentType.STOCK_OPTION: 2>
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from enum.Enum:
 |  
 |  name
 |      The name of the Enum member.
 |  
 |  value
 |      The value of the Enum member.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors inherited from enum.EnumMeta:
 |  
 |  __members__
 |      Return

In [23]:
help(exchange.insert_order)

Help on method insert_order in module optibook.synchronous_client:

insert_order(instrument_id:str, *, price:float, volume:int, side:str, order_type:str='limit') -> int method of optibook.synchronous_client.Exchange instance
    Insert a limit or IOC order on an instrument.
    
    Parameters
    ----------
    instrument_id: str
        The instrument_id of the instrument to insert the order on.
    price: float
        The (limit) price of the order.
    volume: int
        The number of lots in the order.
    side: str
        'bid' or 'ask', a bid order is an order to buy while an ask order is an order to sell.
    order_type: str
        'limit' or 'ioc', limit orders stay in the book while any remaining volume of an IOC that is not immediately
        matched is cancelled.
    
    Returns
    -------
    int
        An order_id which can be used to e.g. delete or amend the limit order later.



In [24]:
help(exchange.get_instruments)

Help on method get_instruments in module optibook.synchronous_client:

get_instruments() -> Dict[str, optibook.common_types.Instrument] method of optibook.synchronous_client.Exchange instance
    Returns all existing instruments on the exchange
    
    Returns
    -------
    typing.Dict[str, Instrument]
        Dict of instrument_id to the instrument definition.

